In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scanpy as sc
import palantir

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, frameon=False)  # low dpi (dots per inch) yields small inline figures
sys.path.insert(0,'/home/users/jjzhu/source_code/aloe/src')
sys.path.insert(0,'./src')

/home/users/jjzhu/.ve/ti/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


scanpy==1.4.4.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.1 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


In [27]:
from io_utils import load_data_from_file
from shared_utils import run_palantir_pseudotime, run_paga_pseudotime
from scipy.stats import spearmanr

def load_data_mtx(data_fn):
    print(data_fn)
    data_dict = load_data_from_file(data_fn, 'pkl') 
    true_lam = data_dict['lam']
    start_index = np.argmin(true_lam)
    index = np.arange(len(true_lam)).astype(str)
    index[start_index] = 'start_cell'
    print('Number of cells: {}'.format(len(true_lam)))
    mtx = np.concatenate([data_dict['z'], data_dict['x']], axis=1)
    n_lm_genes = data_dict['z'].shape[1]
    mtx = pd.DataFrame(mtx, index=index)
    return mtx, true_lam, n_lm_genes

def run_method_and_save_result(mtx, method, fn):
    if method == 'palantir':
        pr_res, out_df = run_palantir_pseudotime(mtx, 'start_cell')
    if method == 'paga':
        adata = sc.AnnData(mtx)
        adata = run_paga_pseudotime(adata, 'start_cell')
        out_df = adata.obs
    out_df.to_csv(fn)
    print('Saved {}'.format(fn))
    return out_df

for regime in range(3):
    ddir = '/share/PI/sabatti/feat_viz/corr_sim/'
    data_name = 'regime_{}'.format(regime)
    data_fn = os.path.join(ddir, data_name, 'data_dict.pkl')
    mtx, true_lam, n_lm_genes = load_data_mtx(data_fn)
    # load the SV genes from GLISS
    rfn = os.path.join(ddir, 'regime_{}'.format(regime), 'method_result.pkl')
    rej_idx = load_data_from_file(rfn, 'pkl')["rejections"]
    # use the LM genes + the SV genes selected from GLISS
    use_idx = list(np.arange(n_lm_genes))  + list(rej_idx + n_lm_genes)
    mtx = mtx[use_idx] # because mtx is a pd.DataFrame()
    print(mtx.shape)
    for method in ['palantir', 'paga']:
        out_fn = 'ti_{}_result_SV_only.csv'.format(method)
        out_fn = os.path.join(ddir, data_name, out_fn)
        out_df = run_method_and_save_result(mtx, method, out_fn)
        # internal evaluation
        corr = abs(spearmanr(out_df['pseudotime'], true_lam).correlation)
        print('Regime {}; {}: {}'.format(regime, method, corr))

/share/PI/sabatti/feat_viz/corr_sim/regime_0/data_dict.pkl
Number of cells: 1500
(1500, 922)
Determing nearest neighbor graph...
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
Sampling and flocking waypoints...
Time for determining waypoints: 0.003908411661783854 minutes
Determining pseudotime...
Shortest path distances using 30-nearest neighbor graph...
Time for shortest paths: 0.05874452988306681 minutes
Iteratively refining the pseudotime...
Correlation at iteration 1: 1.0000
Entropy and branch probabilities...
Markov chain construction...
Identification of terminal states...
Computing fundamental matrix and absorption probabilities...
Project results to all cells...


Transforming to str index.


Saved /share/PI/sabatti/feat_viz/corr_sim/regime_0/ti_palantir_result_SV_only.csv
Regime 0; palantir: 0.9990925969300432
computing PCA with n_comps = 50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.9975364  0.99107105 0.97778195 0.96141744 0.9372904
     0.91699713 0.8799593  0.8471005  0.82443863 0.79078645 0.7401439
     0.7168383  0.67207986 0.65831697]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)
Saved /share/PI/sabatti/feat_viz/

Transforming to str index.


Saved /share/PI/sabatti/feat_viz/corr_sim/regime_1/ti_palantir_result_SV_only.csv
Regime 1; palantir: 0.9986139122728499
computing PCA with n_comps = 50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.9949939  0.98284084 0.9539886  0.921836   0.8703824
     0.83716017 0.7793777  0.73826206 0.72816396 0.7175024  0.7134884
     0.7047496  0.69504577 0.69010943]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)
Saved /share/PI/sabatti/feat_viz/

Transforming to str index.


Saved /share/PI/sabatti/feat_viz/corr_sim/regime_2/ti_palantir_result_SV_only.csv
Regime 2; palantir: 0.9971830103035603
computing PCA with n_comps = 50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.98440903 0.94863325 0.8682447  0.8171155  0.81241107
     0.81110495 0.80337924 0.79680645 0.79377115 0.79140455 0.7869335
     0.78404623 0.78002095 0.7731574 ]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)
Saved /share/PI/sabatti/feat_viz